## Compresión de los datos

In [1]:
import pandas as pd
import dtale as dt

In [2]:
#Global variables in notebook
pd.set_option('display.max_columns', None)
path_data = '../../data/raw/Base.csv'

In [3]:
data = pd.read_csv(path_data)
data.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,13096.035018,7850.955007,6742.080561,5,5,CB,163,1,BC,0,1,9,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,9223.283431,5745.251481,5941.664859,3,18,CA,154,1,BC,1,1,2,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,4471.472149,5471.988958,5992.555113,15,11,CA,89,1,BC,0,1,30,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,14431.993621,6755.344479,5970.336831,11,13,CA,90,1,BC,0,1,1,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,7601.511579,5124.046930,5940.734212,1,6,CA,91,0,BC,1,1,26,0,200.0,0,INTERNET,3.743048,other,0,1,0,0


## Descripción de los datos 

In [4]:
print(data.shape)

(1000000, 32)


In [5]:
# Verificar si los índices son únicos
print("Verificar si los índices son únicos: ",data.index.is_unique)
registros, columnas = (data.shape)
print("El dataset contiene {registros} registros y {columnas} columnas.".format(registros=registros,columnas=columnas))
# Se imprime todas las columnas del dataset con un salto de línea cada 6 columnas
columnas = data.columns.to_list()
print('Las columnas del dataset son:', end="\n")
for i in range(0, len(columnas), 8):
    print(', '.join(columnas[i:i+8]))

Verificar si los índices son únicos:  True
El dataset contiene 1000000 registros y 32 columnas.
Las columnas del dataset son:
fraud_bool, income, name_email_similarity, prev_address_months_count, current_address_months_count, customer_age, days_since_request, intended_balcon_amount
payment_type, zip_count_4w, velocity_6h, velocity_24h, velocity_4w, bank_branch_count_8w, date_of_birth_distinct_emails_4w, employment_status
credit_risk_score, email_is_free, housing_status, phone_home_valid, phone_mobile_valid, bank_months_count, has_other_cards, proposed_credit_limit
foreign_request, source, session_length_in_minutes, device_os, keep_alive_session, device_distinct_emails_8w, device_fraud_count, month


### Descripción de los datos

| **Columna**                               | **Tipo de dato**                    | **Descripción**                                                                                                           | **Rangos**               |
|-------------------------------------------|-----------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------|--------------------------|
| income                                    | Numérico                                      | Ingreso anual del solicitante (en forma de decil).                                                                                  | [0.1, 0.9]               |
| name_email_similarity                     | Numérico                                      | Métrica de similitud entre el correo electrónico y el nombre del solicitante. Valores más altos representan mayor similitud.        | [0, 1]                   |
| prev_address_months_count                 | Numérico                                      | Número de meses en la dirección anterior registrada del solicitante, es decir, la residencia anterior del solicitante, si aplica.    | [−1, 380]                |
| current_address_months_count              | Numérico                                      | Meses en la dirección registrada actual del solicitante.                                                                            | [−1, 429]                |
| customer_age                              | Numérico                                      | Edad del solicitante en años, redondeada a la década.                                                                               | [10, 90]                 |
| days_since_request                        | Numérico                                      | Número de días transcurridos desde que se realizó la solicitud.                                                                    | [0, 79]                  |
| intended_balcon_amount                    | Numérico                                      | Monto inicial transferido para la solicitud.                                                                                       | [−16, 114]               |
| payment_type                              | Categórico                                    | Tipo de plan de pago de crédito. 5 valores posibles (anonimizados).                                                                  | -                        |
| zip_count_4w                              | Numérico                                      | Número de solicitudes dentro del mismo código postal en las últimas 4 semanas.                                                     | [1, 6830]                |
| velocity_6h                               | Numérico                                      | Velocidad de las aplicaciones totales realizadas en las últimas 6 horas, es decir, el número promedio de solicitudes por hora.      | [−175, 16818]            |
| velocity_24h                              | Numérico                                      | Velocidad de las aplicaciones totales realizadas en las últimas 24 horas, es decir, el número promedio de solicitudes por hora.     | [1297, 9586]             |
| velocity_4w                               | Numérico                                      | Velocidad de las aplicaciones totales realizadas en las últimas 4 semanas, es decir, el número promedio de solicitudes por hora.    | [2825, 7020]             |
| bank_branch_count_8w                      | Numérico                                      | Número total de solicitudes en la sucursal bancaria seleccionada en las últimas 8 semanas.                                         | [0, 2404]                |
| date_of_birth_distinct_emails_4w          | Numérico                                      | Número de correos electrónicos de solicitantes con la misma fecha de nacimiento en las últimas 4 semanas.                           | [0, 39]                  |
| employment_status                         | Categórico                                    | Estado de empleo del solicitante. 7 valores posibles (anonimizados).                                                                | -                        |
| credit_risk_score                         | Numérico                                      | Puntuación interna de riesgo de la solicitud.                                                                                       | [−191, 389]              |
| email_is_free                             | Binario                                       | Dominio del correo electrónico de la solicitud (ya sea gratuito o de pago).                                                          | -                        |
| housing_status                            | Categórico                                    | Estado residencial actual del solicitante. 7 valores posibles (anonimizados).                                                      | -                        |
| phone_home_valid                          | Binario                                       | Validez del teléfono fijo proporcionado.                                                                                            | -                        |
| phone_mobile_valid                        | Binario                                       | Validez del teléfono móvil proporcionado.                                                                                            | -                        |
| bank_months_count                         | Numérico                                      | Cuántos meses tiene la cuenta anterior (si la tiene).                                                                               | [−1, 32]                 |
| has_other_cards                           | Binario                                       | Si el solicitante tiene otras tarjetas de la misma entidad bancaria.                                                               | -                        |
| proposed_credit_limit                     | Numérico                                      | Límite de crédito propuesto para el solicitante.                                                                                   | [200, 2000]              |
| foreign_request                           | Binario                                       | Si el país de origen de la solicitud es diferente al del banco.                                                                     | -                        |
| source                                    | Categórico                                    | Fuente en línea de la solicitud. Ya sea navegador (INTERNET) o aplicación (TELEAPP).                                                  | -                        |
| session_length_in_minutes                 | Numérico                                      | Duración de la sesión del usuario en el sitio web bancario en minutos.                                                              | [−1, 107]                |
| device_os                                 | Categórico                                    | Sistema operativo del dispositivo que realizó la solicitud. Los valores posibles son: Windows, macOS, Linux, X11, u otro.            | -                        |
| keep_alive_session                        | Binario                                       | Opción del usuario sobre el cierre de sesión.                                                                                       | -                        |
| device_distinct_emails                    | Numérico                                      | Número de correos electrónicos distintos en el sitio web bancario desde el dispositivo utilizado en las últimas 8 semanas.            | [−1, 2]                  |
| device_fraud_count                        | Numérico                                      | Número de solicitudes fraudulentas con el dispositivo utilizado.                                                                    | [0, 1]                   |
| month                                     | Numérico                                      | Mes en el que se realizó la solicitud.                                                                                              | [0, 7]                   |
| fraud_bool                                | Binario                                       | Si la solicitud es fraudulenta o no.                                                                                                | -                        |


### Descripción de manera general de los tipos de variable para las columnas del dataframe

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   prev_address_months_count         1000000 non-null  int64  
 4   current_address_months_count      1000000 non-null  int64  
 5   customer_age                      1000000 non-null  int64  
 6   days_since_request                1000000 non-null  float64
 7   intended_balcon_amount            1000000 non-null  float64
 8   payment_type                      1000000 non-null  object 
 9   zip_count_4w                      1000000 non-null  int64  
 10  velocity_6h                       1000000 non-null  float64
 11  velocity_24h                      1000

In [7]:
#Se muestra la cantidad de registros por columa que tienen valores nulos
pd.DataFrame(data.isna().sum()).T.style.background_gradient()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Verificar la cantidad de registros duplicados

In [8]:
data_duplicated = data[data.duplicated()]
data_duplicated

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month


### Descripción de los datos de manera general usando medidas estadisticas
Se muestra los datos como si fueran un mapa da calor para poder identificar de mejor manera los datos extremos para una variable

In [9]:
data.describe(percentiles=[.25, .50, .75, 1]).T.style.background_gradient()

,count,mean,std,min,25%,50%,75%,100%,max
fraud_bool,1000000.000000,0.011029,0.104438,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
income,1000000.000000,0.562696,0.290343,0.100000,0.300000,0.600000,0.800000,0.900000,0.900000
name_email_similarity,1000000.000000,0.493694,0.289125,0.000001,0.225216,0.492153,0.755567,0.999999,0.999999
prev_address_months_count,1000000.000000,16.718568,44.046230,-1.000000,-1.000000,-1.000000,12.000000,383.000000,383.000000
current_address_months_count,1000000.000000,86.587867,88.406599,-1.000000,19.000000,52.000000,130.000000,428.000000,428.000000
customer_age,1000000.000000,33.689080,12.025799,10.000000,20.000000,30.000000,40.000000,90.000000,90.000000
days_since_request,1000000.000000,1.025705,5.381835,0.000000,0.007193,0.015176,0.026331,78.456904,78.456904
intended_balcon_amount,1000000.000000,8.661499,20.236155,-15.530555,-1.181488,-0.830507,4.984176,112.956928,112.956928
zip_count_4w,1000000.000000,1572.692049,1005.374565,1.000000,894.000000,1263.000000,1944.000000,6700.000000,6700.000000
velocity_6h,1000000.000000,5665.296605,3009.380665,-170.603072,3436.365848,5319.769349,7680.717827,16715.565404,16715.565404


### Obtención de metadatos

In [10]:
# Se usa la librería dtale para poder obtener una vista interactiva de los datos

bank_account = dt.show(data)
bank_account.open_browser()